In [28]:
# installing appropiate libraries and packages

%pip install pandas
%pip install numpy
%pip install rouge-score

  Using cached rouge_metric-1.0.1-py3-none-any.whl (151 kB)
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tests/__init__.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' c

In [49]:
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer

In [4]:
# loading full dataset in
movies = pd.read_csv('rotten_tomatoes_movies.csv')
reviews = pd.read_csv('rotten_tomatoes_critic_reviews.csv')

In [5]:
# dropping movies that don't have ground truth
filteredMovies = movies.dropna(subset = "critics_consensus")

In [6]:
# parameter: how many movies to conduct the analysis for (CHANGE THIS)
to_keep = 1000

furtherFilteredMovies = filteredMovies[:to_keep]

In [7]:
movies_to_keep = furtherFilteredMovies[['rotten_tomatoes_link']]

# data table for each critic review that has ground truth
combined_data = movies_to_keep.merge(reviews, how = 'left')

In [8]:
combined_data.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [9]:
# dropping data that don't have review values
filtered_combined_data = combined_data.dropna(subset = "review_content")

In [10]:
# grouping critic reviews by each movie and then appending together each review for a given given
filtered_appended_reviews = \
pd.DataFrame(filtered_combined_data.groupby("rotten_tomatoes_link")['review_content'].apply(lambda x: ','.join(x))).reset_index()

In [11]:
# pulling together a table table that represents each movie and the hypothesis for the summary(joining together of all the reviews for a movie)
final_data = \
furtherFilteredMovies.merge(filtered_appended_reviews, on = 'rotten_tomatoes_link')[['movie_title', 'critics_consensus', 'review_content']]

In [12]:
final_data.head()

,movie_title,critics_consensus,review_content
0,Percy Jackson & the Olympians: The Lightning T...,Though it may seem like just another Harry Pot...,A fantasy adventure that fuses Greek mythology...
1,Please Give,Nicole Holofcener's newest might seem slight i...,"Like Holofcener's previous pictures, Please Gi..."
2,10,Blake Edwards' bawdy comedy may not score a pe...,10 (1979) is known for its numerical rating sy...
3,12 Angry Men (Twelve Angry Men),Sidney Lumet's feature debut is a superbly wri...,"A film with texture, humour and relevance at a..."
4,"20,000 Leagues Under The Sea","One of Disney's finest live-action adventures,...",[The] embodiment of Disney at his best -- fami...


In [48]:
# the human-summarizations of all the reviews for a movie
ground_truth = list(final_data['critics_consensus'])

# the model/our approach at the summarizations of all the reviews for a movie
hypotheses = list(final_data['review_content'])

In [55]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

#running the ROUGE evaluation for certain subset of ground truth and hypothesis pairs
num_of_iterations = 100
all_rouges_1_scores = []
all_rouges_2_scores = []
for i in range(num_of_iterations):
    all_scores_obj = scorer.score(hypotheses[i], ground_truth[i])
    all_rouges_1_scores.append(all_scores_obj["rouge1"])
    all_rouges_2_scores.append(all_scores_obj["rouge2"])

In [56]:
# building up metric results for Rouge 1

rouge_1_precisions = []
rouge_1_recall = []
rouge_1_f1 = []
for each_rouge_1_score in all_rouges_1_scores:
    rouge_1_precisions.append(each_rouge_1_score.precision)
    rouge_1_recall.append(each_rouge_1_score.recall)
    rouge_1_f1.append(each_rouge_1_score.fmeasure)

In [58]:
# analyzing rouge 1 results

avg_rouge_1_precision_score = np.average(rouge_1_precisions)
avg_rouge_1_recall_score = np.average(rouge_1_recall)
avg_rouge_1_f1_score = np.average(rouge_1_f1)

print(f'The average precision value for the rouge 1 scores is {avg_rouge_1_precision_score}')
print(f'The average recall value for the rouge 1 scores is {avg_rouge_1_recall_score}')
print(f'The average f1 value for the rouge 1 scores is {avg_rouge_1_f1_score}')

The average precision value for the rouge 1 scores is 0.7911379940249165
The average recall value for the rouge 1 scores is 0.016316084854465805
The average f1 value for the rouge 1 scores is 0.031489730765128024


In [ ]:
# graping rouge 1 metrics

# x_axis = range(1,num_of_iterations)

In [57]:
# building up metric results for Rouge 2

rouge_2_precisions = []
rouge_2_recall = []
rouge_2_f1 = []
for each_rouge_2_score in all_rouges_2_scores:
    rouge_2_precisions.append(each_rouge_2_score.precision)
    rouge_2_recall.append(each_rouge_2_score.recall)
    rouge_2_f1.append(each_rouge_2_score.fmeasure)